# Conexão com o JUSBR 

Este notebook demonstra como autenticar e obter um token de acesso para o portal JUSBR usando a classe `JUSBR_Scraper`.


In [2]:
import juscraper as jus
import json

jusbr = jus.scraper("jusbr")

## Autenticação manual via gov.br

1. Acesse https://www.jus.br
2. Faça login usando o gov.br
3. Entre na página https://portaldeservicos.pdpj.jus.br/consulta (ou clique no botão 'Consultar processo', que aparece após o login)
4. Na nova página, abra a aba Network do navegador (F12 ou 'Inspecionar elemento')
5. Atualize a página (F5 ou ctrl+R ou no botão atualizar)
6. Nas requisições que vão aparecer, procure a requisição que tem nome 'token'. Clique nela.
7. Na tela ao lado, clique em 'Resposta'.
8. Selecione e copie o campo 'access_token' que aparece lá.


In [5]:
import os
from dotenv import load_dotenv
from getpass import getpass

load_dotenv()

access_token = os.environ.get("JUSBR_ACCESS_TOKEN")
if access_token is None:
    # input como senha oculta
    access_token = getpass("Digite o token JWT: ")


In [6]:

jusbr.auth(token=access_token)

True

Alternativa de autenticação: Firefox

Faça o login no firefox e use o código abaixo, que captura os cookies e usa para obter um token JWT.

In [4]:
jusbr.auth_firefox()

KeyError: 'access_token'

## Baixando metadados dos processos

Baixando uma lista de casos:

In [7]:
processos = ['15006452620228260536']

resultados = jusbr.cpopg(processos)

resultados

,processo,numeroProcesso,idCodexTribunal,detalhes
0,15006452620228260536,1500645-26.2022.8.26.0536,None,"{'nivelSigilo': 0, 'siglaTribunal': 'TJSP', 'n..."


In [8]:
result_list = resultados['detalhes'].iloc[0]

print(json.dumps(result_list['tramitacaoAtual']['grau'], indent=2, ensure_ascii=False))

{
  "sigla": "G1",
  "nome": "1° Grau",
  "numero": 1
}


Se quiser visualizar o texto do caso:

## Baixando autos dos processos

In [9]:
df_docs = jusbr.download_documents(resultados)

print(df_docs.shape)
df_docs.head(3)

HTTP Error fetching document ec47221b-c6e7-5034-b0df-1409b7435f1d for process 15006452620228260536 (URL: https://api-processo.data-lake.pdpj.jus.br/processo-api/api/v1/processos/15006452620228260536/documentos/ec47221b-c6e7-5034-b0df-1409b7435f1d/texto?numeroProcesso=15006452620228260536&idDocumento=ec47221b-c6e7-5034-b0df-1409b7435f1d): 500 Server Error: Internal Server Error for url: https://api-processo.data-lake.pdpj.jus.br/processo-api/api/v1/processos/15006452620228260536/documentos/ec47221b-c6e7-5034-b0df-1409b7435f1d/texto?numeroProcesso=15006452620228260536&idDocumento=ec47221b-c6e7-5034-b0df-1409b7435f1d. Response: N/A
Falha ao baixar texto do doc UUID ec47221b-c6e7-5034-b0df-1409b7435f1d(processo 1500645-26.2022.8.26.0536), ou texto vazio.
HTTP Error fetching document 14a090aa-7431-5763-a98b-ebbcc12abe73 for process 15006452620228260536 (URL: https://api-processo.data-lake.pdpj.jus.br/processo-api/api/v1/processos/15006452620228260536/documentos/14a090aa-7431-5763-a98b-ebbcc

(15, 17)


,numero_processo,idCodex,sequencia,nome,tipo,dataHoraJuntada,nivelSigilo,hrefTexto,hrefBinario,texto,_raw_text_api,arquivo,idOrigem,idDocumento,descricao,tipoDocumento,dataJuntada
0,1500645-26.2022.8.26.0536,25240498715,206,Deciso.pdf,"{'codigo': 404, 'nome': 'Decisão', 'idCodex': ...",2025-05-06T17:42:26,PUBLICO,/processos/1500645-26.2022.8.26.0536/documento...,/processos/1500645-26.2022.8.26.0536/documento...,TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO\nCO...,\nTRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO\n...,"{'tipo': 'text/richtext', 'quantidadePaginas':...",PG5SANT_218747833_0,None,None,None,None
1,1500645-26.2022.8.26.0536,24373093626,196,Ato_Ordinatrio.pdf,"{'codigo': 14575, 'nome': 'Ato Ordinatório', '...",2025-03-11T12:39:13,PUBLICO,/processos/1500645-26.2022.8.26.0536/documento...,/processos/1500645-26.2022.8.26.0536/documento...,TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO\nCO...,\nTRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO\n...,"{'tipo': 'text/richtext', 'quantidadePaginas':...",PG5SANT_214809578_0,None,None,None,None
2,1500645-26.2022.8.26.0536,24373143653,194,Ato_Ordinatrio.pdf,"{'codigo': 14575, 'nome': 'Ato Ordinatório', '...",2025-03-11T12:37:13,PUBLICO,/processos/1500645-26.2022.8.26.0536/documento...,/processos/1500645-26.2022.8.26.0536/documento...,TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO\nCO...,\nTRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO\n...,"{'tipo': 'text/richtext', 'quantidadePaginas':...",PG5SANT_214809408_0,None,None,None,None


In [12]:
df_docs

,numero_processo,idCodex,sequencia,nome,tipo,dataHoraJuntada,nivelSigilo,hrefTexto,hrefBinario,texto,_raw_text_api,arquivo,idOrigem,idDocumento,descricao,tipoDocumento,dataJuntada
0,1500645-26.2022.8.26.0536,25240498715,206,Deciso.pdf,"{'codigo': 404, 'nome': 'Decisão', 'idCodex': ...",2025-05-06T17:42:26,PUBLICO,/processos/1500645-26.2022.8.26.0536/documento...,/processos/1500645-26.2022.8.26.0536/documento...,TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO\nCO...,\nTRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO\n...,"{'tipo': 'text/richtext', 'quantidadePaginas':...",PG5SANT_218747833_0,None,None,None,None
1,1500645-26.2022.8.26.0536,24373093626,196,Ato_Ordinatrio.pdf,"{'codigo': 14575, 'nome': 'Ato Ordinatório', '...",2025-03-11T12:39:13,PUBLICO,/processos/1500645-26.2022.8.26.0536/documento...,/processos/1500645-26.2022.8.26.0536/documento...,TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO\nCO...,\nTRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO\n...,"{'tipo': 'text/richtext', 'quantidadePaginas':...",PG5SANT_214809578_0,None,None,None,None
2,1500645-26.2022.8.26.0536,24373143653,194,Ato_Ordinatrio.pdf,"{'codigo': 14575, 'nome': 'Ato Ordinatório', '...",2025-03-11T12:37:13,PUBLICO,/processos/1500645-26.2022.8.26.0536/documento...,/processos/1500645-26.2022.8.26.0536/documento...,TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO\nCO...,\nTRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO\n...,"{'tipo': 'text/richtext', 'quantidadePaginas':...",PG5SANT_214809408_0,None,None,None,None
3,1500645-26.2022.8.26.0536,23597706731,167,Deciso.pdf,"{'codigo': 404, 'nome': 'Decisão', 'idCodex': ...",2025-01-21T17:24:30,PUBLICO,/processos/1500645-26.2022.8.26.0536/documento...,/processos/1500645-26.2022.8.26.0536/documento...,None,None,"{'tipo': 'text/richtext', 'quantidadePaginas':...",PG5SANT_211282457_0,None,None,None,None
4,1500645-26.2022.8.26.0536,19857985572,162,Ato_Ordinatrio.pdf,"{'codigo': 14575, 'nome': 'Ato Ordinatório', '...",2024-04-30T10:09:40,PUBLICO,/processos/1500645-26.2022.8.26.0536/documento...,/processos/1500645-26.2022.8.26.0536/documento...,TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO\nCO...,\nTRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO\n...,"{'tipo': 'text/richtext', 'quantidadePaginas':...",PG5SANT_192471331_0,None,None,None,None
5,1500645-26.2022.8.26.0536,17917151970,128,Despachos.pdf,"{'codigo': 409, 'nome': 'Despacho', 'idCodex':...",2023-11-09T18:39:26,PUBLICO,/processos/1500645-26.2022.8.26.0536/documento...,/processos/1500645-26.2022.8.26.0536/documento...,TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO\nCO...,\nTRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO\n...,"{'tipo': 'text/richtext', 'quantidadePaginas':...",PG5SANT_180536652_0,None,None,None,None
6,1500645-26.2022.8.26.0536,15550543233,116,Sentenas.pdf,"{'codigo': 550, 'nome': 'Sentença', 'idCodex':...",2023-08-31T17:50:51,PUBLICO,/processos/1500645-26.2022.8.26.0536/documento...,/processos/1500645-26.2022.8.26.0536/documento...,TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO\nCO...,\nTRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO\n...,"{'tipo': 'text/richtext', 'quantidadePaginas':...",PG5SANT_175903135_0,None,None,None,None
7,1500645-26.2022.8.26.0536,15529796977,102,Termos_de_Audincia.pdf,"{'codigo': 681, 'nome': 'Termo de Audiência', ...",2023-08-07T14:53:30,PUBLICO,/processos/1500645-26.2022.8.26.0536/documento...,/processos/1500645-26.2022.8.26.0536/documento...,/\nCOMARCA de São Vicente\n1ª Vara Criminal\nR...,\n/\nCOMARCA de São Vicente\n1ª Vara Criminal\...,"{'tipo': 'text/richtext', 'quantidadePaginas':...",PG5SANT_174120447_0,None,None,None,None
8,1500645-26.2022.8.26.0536,15529796877,88,Despachos.pdf,"{'codigo': 409, 'nome': 'Despacho', 'idCodex':...",2023-05-22T16:31:39,PUBLICO,/processos/1500645-26.2022.8.26.0536/documento...,/processos/1500645-26.2022.8.26.0536/documento...,TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO\nCO...,\nTRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO\n...,"{'tipo': 'text/richtext', 'quantidadePaginas':...",PG5SANT_168592685_0,None,None,None,None
9,1500645-26.2022.8.26.0

In [6]:
print(df_docs['texto'][0][:200])

TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO
COMARCA de SÃO PAULO
Foro Central Juizados Especiais Cíveis
2ª Vara do Juizado Especial Cível - Vergueiro
Rua Vergueiro, 835, São Paulo - SP - cep 01504-001 


## Baixando uma lista grande de processos

Se a lista de processos é muito grande, pode ser uma boa prática baixar um processo por vez e salvar os resultados em um arquivo parquet, por exemplo.

No script abaixo, criamos uma pasta dentro da pasta atual, chamada jusbr/cpopg/. Ali dentro, para cada processo, criamos uma pasta com o número do processo e salvamos 2 arquivos: 1 com os metadados daquele processo e 1 com os textos dos documentos. A chave para juntar as duas bases é o número do processo.

In [1]:
import os
import pandas as pd
from tqdm import tqdm
from joblib import Parallel, delayed

processos = ['15006452620228260536']

resultados = jusbr.cpopg(processos)

NameError: name 'jusbr' is not defined

In [ ]:

def download_document(proc_no, jusbr, verbose=True):
    if verbose:
        print(f"Baixando documentos do processo {proc_no}\n")
    
    resultado = jusbr.cpopg([proc_no])
    row = resultado.iloc[0]
    processo = row['processo']
    row_df = pd.DataFrame([row])
    # cria a pasta
    process_path = os.path.join('jusbr/cpopg', processo)
    os.makedirs(process_path, exist_ok=True)
    
    # baixa documentos do processo
    if verbose:
        print(f"Baixando autos do processo {proc_no}\n")
    
    df_docs = jusbr.download_documents(row_df)
    #df_docs = pd.DataFrame()
    # salva documentos
    df_docs.to_parquet(os.path.join(process_path, 'documents.parquet'), index=False)
    # salva metadata
    row_df.to_parquet(os.path.join(process_path, 'metadata.parquet'), index=False)


Parallel(n_jobs=20)(
    delayed(download_document)(proc_no, jusbr)
    for proc_no in processos
)
